# Explore here

In [16]:
# Your code here
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

archivo = 'https://raw.githubusercontent.com/4GeeksAcademy/data-preprocessing-project-tutorial/main/AB_NYC_2019.csv'

df_original = pd.read_csv(archivo)
df=pd.read_csv(archivo)

# Obtener las dimensiones
datos,atributos=df.shape
print(f"Contamos con {datos} cantidad de datos y {atributos} atributos.")
print()
# Obtener información sobre tipos de datos y valores no nulos
df.info()



Contamos con 48895 cantidad de datos y 16 atributos.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 n

- Contamos con un total de 48.895 filas (alojamientos a alquilar) y 16 columnas, distintos atributos de los departamentos.
- Tanto la variable last_review como la variable reviews_per_month tienen más de 10000 valores nulos.